### Corrected A* Pathfinding with Real Penalty Costs

In [ ]:

import heapq

def heuristic_env_aware(node, goal, points, current, neighbor, constraints):
    x1, y1 = points[node]
    x2, y2 = points[goal]
    base_dist = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5 * 1000

    # Optional heuristic penalty (light influence)
    edge = (current, neighbor) if (current, neighbor) in constraints else (neighbor, current)
    terrain = constraints.get(edge, None)

    penalty = 0
    if terrain == 'stairs':
        penalty = 100
    elif terrain == 'slope':
        penalty = 40
    elif terrain == 'lift':
        penalty = 20
    elif terrain == 'kerb_ramp':
        penalty = 10
    elif terrain == 'obstacle':
        penalty = 9999  # essentially avoid it

    return base_dist + penalty

def a_star_env(graph, start, goal, points, constraints):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {node: float('inf') for node in graph.nodes}
    f_score = {node: float('inf') for node in graph.nodes}
    g_score[start] = 0
    f_score[start] = heuristic_env_aware(start, goal, points, start, start, constraints)

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1], g_score[goal]

        for neighbor in graph.neighbors(current):
            edge = (current, neighbor) if (current, neighbor) in constraints else (neighbor, current)
            terrain = constraints.get(edge, None)

            # Apply terrain penalty directly to g_score (real path cost)
            penalty = 0
            if terrain == 'stairs':
                penalty = 100
            elif terrain == 'slope':
                penalty = 40
            elif terrain == 'lift':
                penalty = 20
            elif terrain == 'kerb_ramp':
                penalty = 10
            elif terrain == 'obstacle':
                penalty = 9999

            tentative_g = g_score[current] + graph[current][neighbor]['cost'] + penalty
            if tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic_env_aware(neighbor, goal, points, current, neighbor, constraints)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None, float('inf')
